In [1]:
import os
import json

import sklearn
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

##### 將 NTU 的站點，還有有哪些日期讀取進來

In [5]:
with open("./NTU_station.txt", 'r') as F:
    NTU_station = F.readline().split(",")
many_days = os.listdir("./release")

##### 創建自己處理的資料的資料夾

In [3]:
os.mkdir("./processed data")
for sta in NTU_station:
    os.mkdir(f"./processed data/{sta}")
NTU_station[:5]

##### 將資料改成以站點為主，儲存每個站點在每天的資料

##### 資料的格式為 [年, 月, 日, 星期, 小時, 分鐘, tot, sbi, act, Loss]

下面有利用特別的換算法，可以將日期轉換為星期

In [4]:
MONTH_VALUE = {10:6,11:2,12:4}
# (日號＋月值＋年值＋過4＋世紀值－潤) % 7
#   ?  +  ?  + 23 + 5 + 0 - 0 = 日 + 月 + 28 % 7
# = 日 + 月 % 7 // 10, 11, 12 月月值分別是 6, 2, 4
# 餘 0 代表星期日
# print(f"current sta:{sta}")
def dataTransform(sta:str):
    for day in many_days[:-1]:
        year = int(day[:4])
        month = int(day[4:6])
        date = int(day[6:])
        the_day = (date + MONTH_VALUE[month]) % 7
        cooked = []
        err = 0
        with open(f"./release/{day}/{sta}.json") as F:
            data = json.load(fp=F)
        for key, val in data.items():
            hour = int(key[:2])
            minute = int(key[3:])
            try:
                cooked.append([year,month,date,the_day,hour,minute,val["tot"],val["sbi"],int(val["act"]),val["Loss"]])
            except KeyError:
                err = 1
                break
        if err == 0:
            np.savetxt(f"./processed data/{sta}/{day}",cooked,"%d %d %d %d %d %d %d %d %d %d")
        else:
            print(day,sta)
            break

In [ ]:
for sta in NTU_station:
    # sta = '500101123'
    dataTransform(sta)